In [1]:
import sys
import os
sys.path.append(os.path.abspath("../.."))
import numpy as np
import yfinance as yf
import plotly.express as px
from tinyshift.series import *
from tinyshift.stats import rolling_window, expanding_window, jacknife, mad
from tinyshift.series import trend_significance
from numpy.random import standard_normal

In [2]:
ticker = 'PETR4.SA'
period = '2y'
data = yf.download(ticker, period=period, interval="1d", auto_adjust=True)
t = np.linspace(0, 10, 1000)
pure_sine = np.sin(2 * np.pi * 1 * t)  # Perfectly forecastable

[*********************100%***********************]  1 of 1 completed


In [3]:
r = standardize_returns(data["Close"]["PETR4.SA"], log=False)

# RSI

In [4]:
vol_rsi = relative_strength_index(data["Close"]["PETR4.SA"], rolling_window=14)

In [5]:
px.line(vol_rsi, title=f"{ticker} Closing Prices").show(fig_type="png")

# Entropy Volatility

In [6]:
vol_ent = rolling_window(r, window_size=60, func=sample_entropy, m=1)

In [7]:
px.line(vol_ent, title=f"{ticker} Closing Prices").show(fig_type="png")

In [8]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    y=data["Close"]["PETR4.SA"],
    x=data.index,
    name="Preço PETR4.SA",
    yaxis="y1"
))

fig.add_trace(go.Scatter(
    x=data.index,
    y=vol_ent,
    name="Volatility Entropy",
    yaxis="y2"
))

fig.add_trace(go.Scatter(
    x=data.index,
    y=vol_rsi,
    name="Volatility RSI",
    yaxis="y3"
))


fig.update_layout(
    yaxis3=dict(
        title="Volatility RSI",
        anchor="free",
        overlaying="y",
        side="right",
        position=1.0,
        showgrid=False
    )
)

fig.update_layout(
    title="Preço PETR4.SA e Volatility Entropy",
    xaxis_title="Data",
    yaxis=dict(title="Preço PETR4.SA"),
    yaxis2=dict(title="Volatility Entropy", overlaying="y", side="right"),
    legend=dict(x=0.01, y=0.99)
)

fig.show(fig_type="png")

# Bollinger's Band

In [9]:
np.sum(bollinger_bands(data["Close"]["PETR4.SA"]))

np.int64(55)

In [10]:
np.sum(bollinger_bands(data["Close"]["PETR4.SA"], center=np.median, spread=mad))

np.int64(169)

# Hampel Filter

In [ ]:
hf = hampel_filter(data["Close"]["PETR4.SA"], window_size=7)

In [12]:
data["Close"]["PETR4.SA"][hf]

Date
2023-11-30    27.144901
2024-01-25    29.692333
2024-01-26    30.206356
2024-02-28    30.561636
2024-02-29    30.342421
2024-03-08    27.742079
2024-03-11    26.948364
2024-03-21    26.986164
2024-04-10    29.926668
2024-05-09    33.804356
2024-05-15    31.151602
2024-05-16    30.243013
2024-05-17    29.764383
2024-05-28    30.664856
2024-07-22    31.638645
2024-07-23    31.229816
2024-07-30    30.579023
2024-08-26    33.954933
2024-08-27    33.500141
2024-08-28    33.980671
2024-11-01    30.393829
2024-11-08    31.045984
2024-11-11    31.106049
2024-11-12    31.689552
2025-02-12    33.606377
2025-02-13    33.643429
2025-02-27    33.912060
2025-02-28    33.282173
2025-03-05    32.068707
2025-03-14    32.883858
2025-03-21    34.088055
2025-03-24    34.041744
2025-03-25    34.310371
2025-04-03    33.347012
2025-04-04    32.003868
2025-04-07    30.734829
2025-04-30    28.466911
2025-05-05    28.153671
2025-05-22    29.738857
2025-05-30    29.330694
2025-06-04    28.698929
2025-06-05 

# ADI-CV Framework

In [13]:
adi_cv(r)

(np.float64(1.0), np.float64(nan))

# Sampen

In [14]:
sample_entropy(data["Close"]["PETR4.SA"], m=3, detrend=True)

np.float64(0.6053143519017568)

In [15]:
sample_entropy(pure_sine, m=2)

np.float64(0.16791509477225003)

# Permutation Entropy

In [16]:
permutation_entropy(data["Close"]["PETR4.SA"], m=3, delay=1, normalize=True)

np.float64(0.9659497797080591)

In [17]:
permutation_entropy(data["Close"]["PETR4.SA"], m=3, delay=1, normalize=False)

np.float64(2.496943958125194)

# Stability Index

In [18]:
stability_index(data["Close"]["PETR4.SA"], detrend=True)

np.float64(0.5231487570695778)

In [19]:
stability_index(pure_sine)

np.float64(0.8656234267494547)

# Theorical Limit

In [20]:
theoretical_limit(data["Close"]["PETR4.SA"], m=3)

np.float64(0.034050220291940936)

In [21]:
theoretical_limit(pure_sine, m=4, delay=1)

np.float64(0.7114310780157527)

# Foreca

In [22]:
foreca(data["Close"]["PETR4.SA"])

0.5866504776414962

In [43]:
rolling_window(data["Close"]["PETR4.SA"], window_size=14, func=foreca)

array([0.3878012 , 0.3878012 , 0.3878012 , 0.3878012 , 0.3878012 ,
       0.3878012 , 0.3878012 , 0.3878012 , 0.3878012 , 0.3878012 ,
       0.3878012 , 0.3878012 , 0.3878012 , 0.3878012 , 0.59369985,
       0.66895176, 0.57612879, 0.53239518, 0.28516194, 0.33020777,
       0.13608149, 0.22749629, 0.21625276, 0.22032984, 0.18335708,
       0.30472985, 0.47644409, 0.51505242, 0.52136699, 0.52810043,
       0.52440259, 0.57274166, 0.58394044, 0.68972013, 0.6947915 ,
       0.6439686 , 0.59103362, 0.41214435, 0.38503917, 0.55886282,
       0.42381685, 0.24027459, 0.30697844, 0.41158212, 0.45943251,
       0.60089739, 0.69109588, 0.64270782, 0.50755078, 0.30632636,
       0.47052805, 0.47995625, 0.43549817, 0.45087799, 0.4386542 ,
       0.5639468 , 0.28895741, 0.22590447, 0.25586917, 0.28145202,
       0.49486316, 0.6154628 , 0.65838672, 0.68011084, 0.66672881,
       0.5630103 , 0.43031693, 0.4105373 , 0.45508431, 0.37088302,
       0.26131006, 0.23067508, 0.34746529, 0.28417555, 0.28525

In [42]:
expanding_window(data["Close"]["PETR4.SA"], window_size=14, func=foreca)

array([0.3878012 , 0.3878012 , 0.3878012 , 0.3878012 , 0.3878012 ,
       0.3878012 , 0.3878012 , 0.3878012 , 0.3878012 , 0.3878012 ,
       0.3878012 , 0.3878012 , 0.3878012 , 0.3878012 , 0.43707412,
       0.45913303, 0.4973164 , 0.54400762, 0.49986893, 0.53617061,
       0.43112494, 0.47240659, 0.50256304, 0.55336747, 0.55688529,
       0.60325703, 0.56442798, 0.54378704, 0.54570451, 0.48223303,
       0.37023724, 0.32504013, 0.29378428, 0.30853569, 0.31728493,
       0.3857963 , 0.41601967, 0.41762424, 0.42662384, 0.45340554,
       0.44450843, 0.39857728, 0.41473876, 0.43191334, 0.45793097,
       0.49349903, 0.52285037, 0.53582031, 0.53682322, 0.52517012,
       0.53581811, 0.55469428, 0.56491459, 0.58179435, 0.58718994,
       0.58167496, 0.58598741, 0.54497347, 0.50683232, 0.48243038,
       0.48654387, 0.49098999, 0.48438292, 0.50469599, 0.51697483,
       0.52759128, 0.5227416 , 0.53433536, 0.55049297, 0.57111706,
       0.5589799 , 0.55484211, 0.55023724, 0.55935212, 0.56308

In [35]:
foreca(pure_sine)

0.9993976808641186

In [36]:
rolling_window(pure_sine, rolling_window=60, func=foreca)

TypeError: foreca() got an unexpected keyword argument 'rolling_window'

In [ ]:
rolling_window(pure_sine, rolling_window=60, func=foreca)

# Hurst's Exponent

In [ ]:
hurst_exponent(r, d=0)

(nan, nan)

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    y=data["Close"]["PETR4.SA"],
    x=data.index,
    name="Preço PETR4.SA",
    yaxis="y1"
))


fig.show(fig_type="png")

In [ ]:
np.random.seed(42)
n_points = 1000
trend = np.linspace(0, 10, n_points)
noise_trend = np.cumsum(standard_normal(n_points) * 0.1)
mean_reversion = np.zeros(n_points)
for t in range(1, n_points):
    mean_reversion[t] = mean_reversion[t-1] * 0.6 + standard_normal() * 0.5
brownian = np.cumsum(standard_normal(n_points))

In [ ]:
hurst_exponent(noise_trend)

(0.6419241912818302, 0.000952023886777138)

In [ ]:
hurst_exponent(mean_reversion)

(0.3685698852341455, 0.05402692976986088)

In [ ]:
hurst_exponent(pure_sine)

(0.8575205638768405, 0.0014654668633357023)